### Collegamento del google drive

In [8]:
from google.colab import files

uploaded = files.upload()


Saving auffusion.py to auffusion.py
Saving auffusion_converter.py to auffusion_converter.py


### Caricamento degli specifici file
Fare attenzione, quando si esegue, assicurarsi di aver collegato la cartella corretta

In [3]:
!touch "/content/.project-root"

### Libreria da installare (Attenzione che dopo l'installazione bisogna riavviare, non andrà perso nulla ma è richiesto)

In [9]:
!pip install lightning
!pip install --upgrade huggingface_hub
!pip install rootutils

In [19]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

31055

In [ ]:
import os
os.kill(os.getpid(), 9)

### Codice effettivo di utilizzo della rete implementata

In [20]:
import importlib
import sys

# Se già importata, rimuovila dal sys.modules per sicurezza
if 'auffusion' in sys.modules:
    del sys.modules['auffusion']

# Ora importa normalmente
import auffusion

# E ricaricala per sicurezza
importlib.reload(auffusion)

# Se già importata, rimuovila dal sys.modules per sicurezza
if 'spettrogram' in sys.modules:
    del sys.modules['spettrogram']

# Ora importa normalmente
import spettrogram

# E ricaricala per sicurezza
importlib.reload(auffusion)


# Ora puoi accedere a tutto come auffusion.AuffusionGuidance ecc.
from auffusion import AuffusionGuidance
import matplotlib.pyplot as plt
import torch
import numpy as np
import soundfile as sf
from spettrogram import *
import gc


if __name__=="__main__":
  for n in range(0,12):
    prompt_audio = "birds singing"
    prompt_video = "a painting of snowy mountains, grayscale"

    # Importanza del prompt per l'audio
    guidance_scale_audio=8

    # Importanza del prompt per l'immagine
    guidance_scale_video=10

    # Rigerimenti alle cartelle
    ref_audio="/content/suoni/prime-facts7-nature-sound-mornings-291488.mp3"
    output_path="/content/outputs"

    model = AuffusionGuidance(fp16=True)
    model = model.to(torch.device('cuda'))

    if ref_audio is not None:
        input_spectrogram = spectrify(ref_audio)
        input_spectrogram = prepare_spectrogram_image(input_spectrogram, target_size=(256, 1024), device='cuda')
    else:
        input_spectrogram = None

    print(f"[MAIN]\t-\tGenerando la: [{n}]")
    spect = model.prompt_to_spec(prompt_audio, prompt_video,
                                height=256, width=1024, num_inference_steps=100,
                                device=torch.device('cuda'), guidance_scale_video=guidance_scale_video, input_spectrogram=input_spectrogram,
                                guidance_scale_audio=guidance_scale_audio)

    print(torch.mean(spect), torch.min(spect), torch.max(spect))
    img = spect.detach().cpu().squeeze(0)
    img = img.permute(1, 2, 0).numpy()
    # img = img.astype("16")
    img = img.clip(0, 1)
    imgs = (img * 255).round().astype('uint8')
    # print(img.shape, type(img))
    # gray = 0.2989 * img[0] + 0.5870 * img[1] + 0.1140 * img[2]

    name_audio = prompt_audio.split(' ')[0] + '_' + prompt_audio.split(' ')[1]
    name_video = prompt_video.split(' ')[4] + '_' + prompt_video.split(' ')[5]

    audio = model.spec_to_audio(spect.squeeze(0))
    sf.write(f'{output_path}/{name_audio}_{n}.wav', np.ravel(audio), samplerate=16000)
    plt.figure()
    plt.imsave(f'{output_path}/{name_video}_{n}.png', img)
    # plt.show()
    # capito l'errore. Dobbiamo mettere fp16=False per usare doppia precisione, ma la GPU non regge,

    torch.cuda.empty_cache()
    gc.collect()


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--auffusion--auffusion-full-no-adapter/snapshots/a75ebf2e22d76f8b3cdafcd2a42fd4abba43f7ec/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--auffusion--auffusion-full-no-adapter/snapshots/a75ebf2e22d76f8b3cdafcd2a42fd4abba43f7ec/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--auffusion--auffusion-full-no-adapter/snapshots/a75ebf2e22d76f8b3cdafcd2a42fd4abba43f7ec/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--auffusion--auffusion-full-no-adapter/snapshots/a75ebf2e22d76f8b3cdafcd2a42fd4abba43f7ec/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay':

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.11/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (513) may be set too low.
  warnings.warn(


[MAIN]	-	Generando la: [0]


RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [ ]:
cmd_wav = f'cp "/content/{name_audio}.wav" "/content/drive/MyDrive/Progetto_ESM/Prove/{name_audio}.wav"'
cmd_png = f'cp "/content/{name_audio}.png" "/content/drive/MyDrive/Progetto_ESM/Prove/{name_audio}.png"'

!{cmd_wav}
!{cmd_png}